In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn import tree
from IPython.display import Image
import pydotplus
import graphviz
import time

In [2]:
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc','verification_status',
       'pymnt_plan', 'title', 'dti', 'delinq_2yrs', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'avg_cur_bal', 'chargeoff_within_12_mths', 'delinq_amnt',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
       'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m',
       'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',
       'pct_tl_nvr_dlq', 'percent_bc_gt_75',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit','id','member_id','funded_amnt','funded_amnt_inv','installment','issue_d',
        'purpose','next_pymnt_d','application_type','inq_last_6mths','acc_open_past_24mths',
        'bc_open_to_buy','bc_util','mo_sin_old_il_acct'], 1, inplace=True)

# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [4]:
pd.get_dummies(y2015)

,loan_amnt,int_rate,annual_inc,out_prncp,last_pymnt_amnt,total_rev_hi_lim,pub_rec_bankruptcies,tax_liens,term_ 36 months,term_ 60 months,...,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days)
0,16000.0,14.85,48000.0,13668.88,379.39,284700.0,0.0,2.0,0,1,...,1,0,0,0,1,0,0,0,0,0
1,9600.0,7.49,60000.0,6635.69,298.58,13000.0,0.0,0.0,1,0,...,1,0,0,0,1,0,0,0,0,0
2,25000.0,7.49,109000.0,0.00,20807.39,38400.0,0.0,0.0,1,0,...,1,0,0,0,0,0,1,0,0,0
3,28000.0,6.49,92000.0,19263.77,858.05,79900.0,0.0,0.0,1,0,...,1,0,0,0,1,0,0,0,0,0
4,8650.0,19.89,55000.0,0.00,8251.42,20750.0,1.0,0.0,1,0,...,0,0,1,0,0,0,1,0,0,0
5,23000.0,8.49,64000.0,19143.69,471.77,47300.0,0.0,0.0,0,1,...,0,0,1,0,1,0,0,0,0,0
6,29900.0,12.88,65000.0,25346.66,678.49,67800.0,0.0,0.0,0,1,...,1,0,0,0,1,0,0,0,0,0
7,18000.0,11.99,112000.0,0.00,18004.90,82400.0,0.0,0.0,0,1,...,1,0,0,0,0,0,1,0,0,0
8,35000.0,14.85,110000.0,29900.89,829.90,67300.0,0.0,0.0,0,1,...,1,0,0,0,1,0,0,0,0,0
9,10400.0,22.45,104433.0,0.00,10128.96,34000.0,0.0,0.0,0,1,...,1,0,0,0,0,0,1,0,0,0


In [12]:
# Model with RandomForestClassifier and measure Runtime
start_time = time.time()
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)
print(cross_val_score(rfc, X, Y, cv=10))

print("--- %s seconds ---" % (time.time() - start_time))

[ 0.94476503  0.9519128   0.94982309  0.95065422  0.95131798  0.94986939
  0.95055689  0.95022442  0.94993707  0.9487484 ]
--- 155.35951805114746 seconds ---


In [13]:
# Model with DecisionTreeClassifier and measure Runtime
start_time = time.time()
decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=1,
    max_depth=5
)

X = y2015.drop('loan_status', 1)
X = pd.get_dummies(X)
X = X.dropna(axis=1)

Z = pd.DataFrame()
Z['loan_status'] = y2015['loan_status']

print(cross_val_score(decision_tree, X, Z, cv=10))

print("--- %s seconds ---" % (time.time() - start_time))

[ 0.69316806  0.83541118  0.69000974  0.76324476  0.68252197  0.69886013
  0.68341685  0.73960436  0.68570546  0.71369876]
--- 18.715515851974487 seconds ---
